In [ ]:

import pandas as pd
import numpy as np
import joblib
import tensorflow as tf

# Load models
nn_model = tf.keras.models.load_model("neural_net_model.h5")
xgb_model = joblib.load("xgboost_model.pkl")
lgb_model = joblib.load("lightgbm_model.pkl")
meta_learner = joblib.load("meta_learner.pkl")
preprocessor = joblib.load("preprocessor.pkl") 

# Load new dataset
df_new = pd.read_csv("Synthetic_SBA_Loans_test.csv")

# Drop unused columns (must match training logic)
drop_cols = ['Applicant ID', 'Country', 'Location', 'NAICS', 'Eligibility Score']
df_new_cleaned = df_new.drop(columns=drop_cols, errors='ignore')

# Preprocess the data
X_new_preprocessed = preprocessor.transform(df_new_cleaned)

# Get probability predictions
nn_probs = nn_model.predict(X_new_preprocessed).flatten()
xgb_probs = xgb_model.predict_proba(X_new_preprocessed)[:, 1]
lgb_probs = lgb_model.predict_proba(X_new_preprocessed)[:, 1]

# Stack model outputs and use meta-learner
meta_X = np.column_stack((nn_probs, xgb_probs, lgb_probs))
final_preds = meta_learner.predict(meta_X)

# Add predictions to the original dataset
df_new["Predicted_Eligibility"] = final_preds

# Save the results
df_new.to_csv("Predicted_SBA_Loans.csv", index=False)

print("Prediction complete. Output saved to 'Predicted_SBA_Loans.csv'.")
